In [1]:
import numpy as np
from collections import Counter


In [139]:
def split_random(size , p) :
    #sep_key = ["train","valid","test"][:len(p)]
    sep_key = [0,1,2][:len(p)]
    return np.random.choice(sep_key,size=size , p=p)

from collections import namedtuple
SEPIDX = namedtuple("Seperate", ["target" , "idx","split_idx"])

class SplitIndex(object) :
    def __init__(self,arr) :
        self.arr = arr 
        sep_key = ["train","valid","test"]
        self.idx_dict = {idx:sep  for idx , sep in enumerate(sep_key)}
    
    def __call__(self,p , mode="random") :
        self.p = p
        assert np.isclose(np.sum(p),1) , "sum of p is 1"
        if mode == "random" :
            V = split_random(len(self.arr), self.p)
            a = np.array([np.arange(len(target)) , V]).T
        elif mode == "stratified" :
            a = self._split_stratified(self.arr)
        self.a = self._sort_by_idx(a) 
        return self.a 

    def _sort_by_idx(self, a ) :
        a = a[a[:, 0].argsort()]
        return a 

    def _split_stratified(self,arr) :
        result = []
        for i in np.unique(arr) :
            idxs = np.where(arr==i)[0]
            split_idx = split_random(len(idxs),self.p)
            result.append(SEPIDX(target=i , idx = idxs , split_idx = split_idx))
        idxs = []
        split_idxs = []
        for namedtuple in result :
            idxs.extend(namedtuple.idx)
            split_idxs.extend(namedtuple.split_idx)
        a = np.array([idxs , split_idxs]).T
        
        return a

    def print_count(self,) :
        s , k , c, ss = "split", "target", "count" , "sum"
        print(f"|{s:10s}|{k:10s}|{c:10s}|{ss:10s}|")
        print("| :------- | :------- | :------- | :------- |")
        for sep_idx in np.unique(self.a[:,1]) :
            idxs = np.where(self.a[:,1]==sep_idx)
            self.prettyproint_counter(self.idx_dict[sep_idx] , Counter(self.arr[idxs]))

    def prettyproint_counter(self,name , Counter) :

        dict_ = dict(Counter)
        sorted_key_value = sorted(dict_.items(), key = lambda key_value : key_value[0])
        for idx , (key, value) in enumerate(sorted_key_value):
            if idx == 0 :
                s = int(np.sum(list(Counter.values())))
                print(f"|{name:10s}|{str(key):10s}|{str(value):10s}|{str(s):10s}|")
            else :
                s = ""
                print(f"|{name:10s}|{str(key):10s}|{str(value):10s}|{s:10s}|")


|split     |target    |count     |sum       |
| :------- | :------- | :------- | :------- |
|train     |0         |44        |62        |
|train     |1         |18        |          |
|valid     |0         |17        |38        |
|valid     |1         |21        |          |

In [150]:
target = np.random.choice([0,1,2],size=1000 , p=[0.6,0.3,0.1])
split_idx = SplitIndex(target)
idx_split_value = split_idx(p=[0.6,0.3,0.1],mode="stratified")

In [151]:
split_idx.print_count()


|split     |target    |count     |sum       |
| :------- | :------- | :------- | :------- |
|train     |0         |357       |592       |
|train     |1         |171       |          |
|train     |2         |64        |          |
|valid     |0         |173       |296       |
|valid     |1         |99        |          |
|valid     |2         |24        |          |
|test      |0         |68        |112       |
|test      |1         |35        |          |
|test      |2         |9         |          |


In [152]:
idx_split_value

array([[  0,   2],
       [  1,   1],
       [  2,   0],
       ...,
       [997,   0],
       [998,   0],
       [999,   1]])

[0, 1, 2]